In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from  tensorflow.keras.applications import ResNet50, vgg16
from  tensorflow.keras.models import Sequential, Model
from  tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D
from  tensorflow.keras import optimizers
from tensorflow import keras
import tensorflow as tf
import datetime
#画像サイズ
imagesize = 96
#データの前処理
image_data_genelator = keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255,shear_range=0.2,
   zoom_range=0.2,
   horizontal_flip=True,
   rotation_range=10)
 
train_generator = image_data_genelator.flow_from_directory(
    "./mask-model",
    target_size = (imagesize,imagesize),
    color_mode='rgb',
    classes = ["model-H","model-S","model-U"],
    class_mode='categorical',
    batch_size = 4,
    shuffle=True
)

test_generator = image_data_genelator.flow_from_directory(
    "./test-mask-model",
    target_size = (imagesize,imagesize),
    color_mode='rgb',
    classes = ["model-H","model-S","model-U"],
    class_mode='categorical',
    batch_size = 4,
    shuffle=True
)
#モデルの定義
input_tensor = Input(shape=(imagesize, imagesize, 3))
base_model = vgg16.VGG16(weights='imagenet', include_top=False,input_tensor=input_tensor)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
#callback関数の定義
for layer in base_model.layers:
    layer.trainable = False
model.compile(loss='categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping()

checkpoint = keras.callbacks.ModelCheckpoint(filepath ="./models/RCJ_Image_Recognition/model{epoch:2d}.pb",
                                                                                 monitor='val_loss',
                                                                                 save_best_only=False,
                                                                                 save_weights_only=False,
                                                                                 include_optimizer = False,
                                                                                 mode='min',)

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
#コマンドラインで、"tensorboard --logdir logs/fit"と入力しTensorboardを起動
#学習
history = model.fit(
    train_generator ,
    validation_data=test_generator ,
    epochs=10,
    verbose=1,
    callbacks=[early_stopping,checkpoint,tensorboard_callback])

Found 446 images belonging to 3 classes.
Found 608 images belonging to 3 classes.
Epoch 1/10
112/112 [==============================] - ETA: 0s - loss: 0.6811 - accuracy: 0.7130INFO:tensorflow:Assets written to: ./models/RCJ_Image_Recognition/model 1.pb/assets


INFO:tensorflow:Assets written to: ./models/RCJ_Image_Recognition/model 1.pb/assets


112/112 [==============================] - 30s 259ms/step - loss: 0.6811 - accuracy: 0.7130 - val_loss: 0.4713 - val_accuracy: 0.7961
Epoch 2/10
112/112 [==============================] - ETA: 0s - loss: 0.2435 - accuracy: 0.9305INFO:tensorflow:Assets written to: ./models/RCJ_Image_Recognition/model 2.pb/assets


INFO:tensorflow:Assets written to: ./models/RCJ_Image_Recognition/model 2.pb/assets


112/112 [==============================] - 30s 273ms/step - loss: 0.2435 - accuracy: 0.9305 - val_loss: 0.3320 - val_accuracy: 0.8799
Epoch 3/10
112/112 [==============================] - ETA: 0s - loss: 0.1488 - accuracy: 0.9552INFO:tensorflow:Assets written to: ./models/RCJ_Image_Recognition/model 3.pb/assets


INFO:tensorflow:Assets written to: ./models/RCJ_Image_Recognition/model 3.pb/assets


112/112 [==============================] - 30s 267ms/step - loss: 0.1488 - accuracy: 0.9552 - val_loss: 0.3457 - val_accuracy: 0.8651


In [65]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_saved_model("./models/RCJ_Image_Recognition/model06.pb")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_model = converter.convert()

with open("model.tflite", "wb") as f:
    f.write(tflite_model)
    

2023-08-07 20:40:50.016437: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-08-07 20:40:50.016478: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-08-07 20:40:50.016794: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: ./models/RCJ_Image_Recognition/model06.pb
2023-08-07 20:40:50.019650: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2023-08-07 20:40:50.019667: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: ./models/RCJ_Image_Recognition/model06.pb
2023-08-07 20:40:50.027085: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-08-07 20:40:50.145997: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: ./models/RCJ_Image_Recognition/model06.pb
2023-08-07 20:40:50.170460: I tensorflow/cc/saved_model/loader.cc

In [66]:
import numpy as np
import tensorflow as tf
import time
import cv2
from PIL import Image

start_time = time.time()
image_size = 96
classes = ["H","S","U"]

interpreter = tf.lite.Interpreter(model_path="./model.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_shape = input_details[0]['shape']

image = Image.open("model627.jpg")
image = image.convert('RGB')
image = image.resize((image_size, image_size))
data = np.asarray(image)/255
X = []
X.append(data)
X = np.array(X, dtype=np.float32)

interpreter.set_tensor(input_details[0]['index'], X)

# 推論実行
interpreter.invoke()

output_data = interpreter.get_tensor(output_details[0]['index'])
print(classes[np.argmax(output_data)])

H
